In [1]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

# Метод прогонки

Формулы прогоночных коэффициентов:

$P_1 = \frac{-c_1}{b_1}, P_i = \frac{-c_i}{b_i + a_i P_{i-1}}$

$Q_1 = \frac{d_1}{b_1}, Q_i = \frac{d_i - a_i Q_{i-1}}{b_i + a_i P_{i-1}}$

Формулы для поиска решений: 

$x_n = P_n x_{n + 1} + Q_n$

In [2]:
matrix_array = np.loadtxt('src/sweep.txt')
print(matrix_array)
shape = matrix_array.shape

[[-10.  -9.   0.   0.   0.   7.]
 [ -5. -21.  -8.   0.   0.  29.]
 [  0.   7.  12.   2.   0.  31.]
 [  0.   0.   0.   8.   2.  56.]
 [  0.   0.   0.   2.  10. -24.]]


In [3]:
def solve_sweep(matrix):
    p = []
    q = []

    p.append(-c_i(matrix, 0) / b_i(matrix, 0))
    q.append(d_i(matrix, 0) / b_i(matrix, 0))

    for i in range(1, shape[0]):
        p.append(p_i(matrix, i, p))
        q.append(q_i(matrix, i, p ,q))

    answer = [0] * shape[0]
    answer[-1] = q[-1]

    for i in range(shape[0] - 2, -1, -1):
        answer[i] = x_i(i, p, q, answer)

    print("Коэффициенты P:")
    print(p)
    print("Коэффициенты Q:")
    print(q)
    print("Решение:")
    return np.array(answer)

def p_i(matrix, i, p):
    return -c_i(matrix, i) / (b_i(matrix, i) + a_i(matrix, i) * p[i - 1])

def q_i(matrix, i, p ,q):
    return (d_i(matrix, i) - a_i(matrix, i) * q[i - 1]) / (b_i(matrix, i) + a_i(matrix, i) * p[i - 1])

def x_i(i, p, q, answer):
    return p[i] * answer[i + 1] + q[i]

def a_i(matrix, i):
    if i == 0:
        return 0
    return matrix[i, i - 1]

def b_i(matrix, i):
    return matrix[i, i]

def c_i(matrix, i):
    if i == (shape[0] - 1):
        return 0
    return matrix[i, i + 1]

def d_i(matrix, i):
    return matrix[i, -1]

In [4]:
print(solve_sweep(matrix_array))

Коэффициенты P:
[-0.9, -0.48484848484848486, -0.23239436619718312, -0.25, 0.0]
Коэффициенты Q:
[-0.7, -1.5454545454545454, 4.859154929577465, 7.0, -4.0]
Решение:
[ 2. -3.  3.  8. -4.]


# Метод итераций

Основан на приведении исходного уравнения $Ax = B$ к эквивалентному виду $x = \alpha x + \beta$

Один из способов построения матриц $\alpha$, $\beta$ (метод Якоби):

$\alpha_{ij} = -\frac{a_{ij}}{a_{ii}} (\alpha_{ii} = 0), \beta_i = \frac{b_i}{a_{ii}}$

В таком случае решения ищутся по следующему итерационному процессу:

$x^{(k)} = \beta + \alpha x^{(k - 1)}$

In [5]:
matrix_array = np.loadtxt('src/iterations.txt')
print(matrix_array)
shape = matrix_array.shape

[[  21.    1.   -8.    4. -119.]
 [  -9.  -23.   -2.    4.   79.]
 [   7.   -1.  -17.    6.  -24.]
 [   8.    8.   -4.  -26.  -52.]]


In [6]:
def check_diagonal_dominance(matrix):
    shape = matrix.shape
    for row in range (shape[0]):
        row_sum = np.sum(np.abs(matrix), axis=1)
        if np.abs(matrix[row, row]) < (row_sum[row] - np.abs(matrix[row, -1]) - np.abs(matrix[row, row])):
            return False
    return True
    
def solve_iterations(matrix, eps):
    if not check_diagonal_dominance(matrix):
        return
    matrix = get_non_zero_elements_at_diagonal(matrix)

    beta = get_betas_column(matrix)
    alpha = get_alphas_column(matrix)

    x = np.copy(beta)
    iterations = 0
    while True:
        iterations += 1
        x_i = beta + np.matmul(alpha, x)
        if is_accuracy_achieved(x_i, x, eps):
            return x_i.T[0], iterations
        x = x_i

def is_accuracy_achieved(x, x_prev, eps):
    return np.max(np.abs(x - x_prev)) <= eps

def get_betas_column(matrix):
    beta = np.zeros((shape[0], 1))
    for i in range(0, shape[0]):
        beta[i, :] = matrix[i, -1] / matrix[i, i]

    return beta

def get_alphas_column(matrix):
    alpha = np.zeros((shape[0], shape[0]))

    for i in range(0, shape[0]):
        for j in range(0, shape[0]):
            if i == j:
                continue
            else:
                alpha[i, j] = -matrix[i, j] / matrix[i, i]

    return alpha

def get_non_zero_elements_at_diagonal(matrix):
    for column in range (0, shape[0]):
        if matrix[column, column] == 0:
            set_non_zero_element_for_column(matrix, column)

    return matrix

def set_non_zero_element_for_column(matrix, column):
    for row in range (0, shape[0]):
        if matrix[row, column] != 0:
            matrix[[column, row]] = matrix[[row, column]]
            return

In [7]:
x, iterations = solve_iterations(matrix_array, 0.00001)
print("Решение:")
print(x)
print("Итерации: {0}".format(iterations))

Решение:
[-6. -1. -1.  0.]
Итерации: 18


# Метод Зейделя

Отличается от методы итераций тем, что при вычислении очередного приближения решения $x^{(k)}$ считаются не все значения, а часть используются с предыдущей итерации. То есть:

$x^{(k + 1)} = \beta + B x^{(k + 1)} + C x^{(k)}, \alpha = B + C$

In [8]:
def solve_zeidel(matrix, eps):
    if not check_diagonal_dominance(matrix):
        return

    matrix = get_non_zero_elements_at_diagonal(matrix)

    beta = get_betas_column(matrix)
    alpha = get_alphas_column(matrix)

    x = beta
    iterations = 0

    while True:
        iterations += 1
        x_n = np.copy(x)
        for i in range (shape[0]):
             x_n[i] = beta[i] + sum(alpha[i, j] * x_n[j] for j in range(i)) + sum(alpha[i, j] * x[j] for j in range(i, shape[0]))

        if is_accuracy_achieved(x_n, x, eps):
            return x_n.T[0], iterations

        x = x_n

In [9]:
x, iterations = solve_zeidel(matrix_array, 0.00001)
print("Решение:")
print(x)
print("Итерации: {0}".format(iterations))

Решение:
[-6. -1. -1.  0.]
Итерации: 10


# Метод вращений

Используется для нахождения собственных значений и векторов симметрических матриц

На каждой итерации ставится задача обнуления макиммального недиагонального элемента матрицы с помощью операции $A^{(k + 1)} = U^{(k)T} A^{(k)} U^{(K)}$

Матрица $U^{(k)}$ определяется:

$u^{(k)}_{ij} = -\sin(\varphi^{(k)}), u^{(k)}_{ji} = \sin(\varphi^{(k)})$

$u^{(k)}_{ii} = u^{(k)}_{jj} = \cos(\varphi^{(k)})$

$\varphi^{(k)} = \frac{1}{2} \arctan(\frac{2 a^{(k)}_{ij}}{a^{(k)}_{ii} - a^{(k)}_{jj}})$

По окончании итерационного процесса получим диагональную матрицу $\Lambda$, элементы диагонали которой будут собственными значениями исходной матрицы, и матрицы $U^{(0)}, ..., U^{(n - 1)}$ - координатные столбцы их произведения будут собственными векторами исходной матрицы

In [10]:
matrix_array = np.loadtxt('src/rotations.txt')
print(matrix_array)
shape = matrix_array.shape

[[-3. -2. -4.]
 [-2. -3. -7.]
 [-4. -7. -3.]]


In [11]:
def solve_rotations(matrix, eps):
    if not check_symmetry(matrix):
        return

    a = np.copy(matrix)
    rotation_matrices = []
    iterations = 0
    while True:
        i, j = get_position_of_max_non_diagonal_element(a)
        max_element = a[i, j]
        phi = 1/2 * np.arctan(2*max_element / (a[i, i] - a[j, j])) if a[i, i] != a[j, j] else np.pi / 4
        rotation_matrix = build_rotation_matrix((i, j), shape[0], phi)
        a_k = np.matmul(np.matmul(rotation_matrix.T, a), rotation_matrix)
        if is_accuracy_achieved(a_k, eps):
            return np.diagonal(a_k), get_eigenvectors(rotation_matrices), iterations
        else:
            a = a_k
            rotation_matrices.append(rotation_matrix)
            iterations += 1

def check_symmetry(matrix):
    return np.allclose(matrix, matrix.T)


def get_position_of_max_non_diagonal_element(matrix):
    matrix_for_subtract = np.identity(shape[0]) * np.diagonal(matrix)
    matrix = np.abs(matrix - matrix_for_subtract)
    element =  np.max(matrix)
    indexes =  np.where(matrix == element)
    return indexes[0][0], indexes[1][0]

def build_rotation_matrix(position_of_max_element, size, phi):
    rotation_matrix = np.identity(size)
    i, j = position_of_max_element
    rotation_matrix[i, j] = -np.sin(phi)
    rotation_matrix[j, i] = np.sin(phi)
    rotation_matrix[i, i] = rotation_matrix[j, j] = np.cos(phi)
    return rotation_matrix

def is_accuracy_achieved(matrix, eps):
    matrix_for_subtract = np.identity(shape[0]) * np.diagonal(matrix)
    matrix = np.power(matrix - matrix_for_subtract, 2)
    sum_of_elements = np.power(np.sum(matrix) / 2, 0.5)
    return sum_of_elements < eps

def get_eigenvectors(rotation_matrices):
    result = rotation_matrices[0]
    if len(rotation_matrices) > 1:
        for i in range(1, len(rotation_matrices)):
            result = np.matmul(result, rotation_matrices[i])

    return result

In [12]:
eigenvalues, eigenvectors, iterations = solve_rotations(matrix_array, 0.00001)
print("Собственные зачения:")
print(eigenvalues)
print("Собственные векторы:")
for vector in eigenvectors.T:
    print(vector, end=",\n")
print("Итерации: {0}".format(iterations))

Собственные зачения:
[ -1.306 -12.023   4.329]
Собственные векторы:
[ 0.874 -0.466 -0.137],
[0.43  0.611 0.665],
[-0.226 -0.64   0.735],
Итерации: 6


# QR-разложение

Используется для нахождения собственных значений

В основе лежит представление матрицы в виде $A = QR$, где $Q$ - ортогональная матрица, а $R$ - верхняя треугольная

В алгоритме используется преобразование Хаусхолдера:

$H = E - \frac{2}{\nu^T \nu}\nu \nu^T$

В конце итерационного процесса получим матрицу $R$ и матрицу $Q = H^{(1)} ... H^{(n - 1)}$

In [13]:
matrix_array = np.loadtxt('src/qr.txt')
print(matrix_array)
shape = matrix_array.shape

[[ 7.  6. -3.]
 [ 5. -6.  8.]
 [-7.  4. -5.]]


In [14]:
def solve_qr(matrix, eps):
    a = matrix
    iterations = 0
    while True:
        iterations += 1
        h_matrices = []
        r = a
        for column in range(shape[1] - 1):
            v = get_v_vector(r, column)
            h = get_householder_matrix(v)
            h_matrices.append(h)
            r = np.matmul(h, r)

        q = muliply_matrices(h_matrices)
        a = np.matmul(r, q)
        if is_accuracy_achieved(a, eps):
            return np.diagonal(a), iterations

def get_v_vector(matrix, column):
    v = np.zeros(shape[0])
    v[column] =  matrix[column, column] + np.sign(matrix[column, column]) * np.sqrt(np.sum(np.power(matrix[column:, column], 2)))
    v[column + 1:] = matrix[column + 1:, column]
    return np.atleast_2d(v).T

def get_householder_matrix(v):
    e = np.identity(v.shape[0])
    return e - 2 * np.matmul(v, v.T)/np.matmul(v.T, v)

def muliply_matrices(matrices):
    result = matrices[0]
    if len(matrices) > 1:
        for i in range(1, len(matrices)):
            result = np.matmul(result, matrices[i])

    return result

def is_accuracy_achieved(matrix, eps):
    sum_of_subdiagonal_elements = 0
    for i in range(1, shape[0]):
        sum_of_subdiagonal_elements += np.sum(np.power(np.diagonal(matrix, -i), 2))
    return np.sqrt(sum_of_subdiagonal_elements) < eps

In [15]:
values, iterations = solve_qr(matrix_array, 0.00001)
print("Собственные значения:")
print(values)
print("Итерации: {0}".format(iterations))

Собственные значения:
[-13.983   8.907   1.076]
Итерации: 35
